In [13]:
import logging
import os

# LOGGING CONFIG ##############################
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')
###############################################

In [14]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

In [15]:
'''
using the chest x-ray dataset from https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia/
check if the chest x-ray dataset exists
unzip the chest x-ray dataset if it exists
'''
if not os.path.exists('chest_xray'):
    if os.path.exists('chest-xray-pneumonia.zip'):
        logging.info('unzipping the dataset file')
        os.system('unzip chest-xray-pneumonia.zip')
        logging.info('unzipping is done')
        os.system('rm chest-xray-pneumonia.zip')
    else:
        logging.warning('please download the dataset from https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia/')

In [16]:
def create_training_data(dir, data_type, main, sub, size):
    training_data = []
    occurrence = [0, 0, 0]
    for cat in main:
        logging.info('reading images for category {}'.format(cat))
        path =  os.path.join(dir, data_type, cat)
        label = main.index(cat)
        for img in os.listdir(path):
            # read and resize image
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                img_array = cv2.resize(img_array, (size, size))

                # assign label for virus and bateria
                if label != 0:
                    if sub[0] in img:
                        label = 1
                    else:
                        label = 2

                # append to training data
                occurrence[label] += 1
                training_data.append([img_array, label])

            except:
                logging.warn('error reading {}'.format(img))
    
    logging.info('reading images done')
    return training_data, occurrence

            
directory = 'chest_xray'
data_type = 'train'   # choose between train and test
main_category = ['NORMAL', 'PNEUMONIA']
sub_category = ['bacteria', 'virus']
image_size = 200

# create training data with labels: 0:normal, 1:bacterial 2:viral
training_data, occurrence = create_training_data(directory, data_type, main_category, sub_category, image_size)

2020-05-13 21:50:02 INFO     reading images for category NORMAL
2020-05-13 21:50:28 INFO     reading images for category PNEUMONIA
2020-05-13 21:50:51 INFO     reading images done


In [17]:
# number of images with labels 0, 1 and 2
print('Normal:', occurrence[0], 'Bacterial:', occurrence[1], 'Viral:', occurrence[2])
# weight is labels based on their occurence
weight = [float(i)/sum(occurrence) for i in occurrence]
print('Normal:', weight[0], 'Bacterial:', weight[1], 'Viral:', weight[2])

# shuffle the training data otherwise the neural network model will be inefficient
import random 
random.shuffle(training_data)

Normal: 1341 Bacterial: 2530 Viral: 1345
Normal: 0.2570935582822086 Bacterial: 0.48504601226993865 Viral: 0.25786042944785276


In [18]:
# separate features and labels
X = []
y = []
for features, label in training_data:
    X.append(features)
    y.append(label)

In [19]:
# we have to convert a list to a numpy array that is understandable for tensorflow
# -1 means everything in the list, 1 is because the image is gray scale
X = np.array(X).reshape(-1, image_size, image_size, 1)


In [20]:
# save the training data
# you need at least 8GB of ram for this
import pickle
pickle_out = open('trainings/X.pickle', 'wb')
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open('trainings/y.pickle', 'wb')
pickle.dump(y, pickle_out)
pickle_out.close()

In [21]:
# import libraries to create neural networks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [22]:
# read X and y in case we want to re run from here
import pickle
X = pickle.load(open('trainings/X.pickle', 'rb'))
y = pickle.load(open('trainings/y.pickle', 'rb'))

# in case of gray scale image data, we normalize it 
X = X/255.0 

In [25]:
# create the model 
model = Sequential() 

# layer 1
model.add(Conv2D(32, (4,4), strides=(4, 4), input_shape=X.shape[1:]))
model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

# layer 2
model.add(Conv2D(32, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))

# layer 3
model.add(Flatten())
model.add(Dense(64))

# output layer, we use Dense(3) to have 3 labels 0,1,2
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 50, 50, 32)        544       
_________________________________________________________________
activation_4 (Activation)    (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        16416     
_________________________________________________________________
activation_5 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                1179712   
_________________________________________________________________
dense_5 (Dense)              (None, 3)                

In [27]:
model.fit(X, y, batch_size=100, epochs=40, validation_split=0.1)

Train on 4694 samples, validate on 522 samples
Epoch 1/40
4694/4694 [==============================] - 9s 2ms/sample - loss: 0.4307 - acc: 0.8130 - val_loss: 0.4861 - val_acc: 0.7835
Epoch 2/40
4694/4694 [==============================] - 9s 2ms/sample - loss: 0.4087 - acc: 0.8179 - val_loss: 0.4945 - val_acc: 0.7739
Epoch 3/40
4694/4694 [==============================] - 9s 2ms/sample - loss: 0.3967 - acc: 0.8304 - val_loss: 0.4727 - val_acc: 0.7893
Epoch 4/40
4694/4694 [==============================] - 9s 2ms/sample - loss: 0.3608 - acc: 0.8426 - val_loss: 0.4743 - val_acc: 0.7969
Epoch 5/40
4694/4694 [==============================] - 9s 2ms/sample - loss: 0.3427 - acc: 0.8509 - val_loss: 0.4965 - val_acc: 0.7778
Epoch 6/40
4694/4694 [==============================] - 9s 2ms/sample - loss: 0.3208 - acc: 0.8626 - val_loss: 0.5403 - val_acc: 0.7567
Epoch 7/40
4694/4694 [==============================] - 9s 2ms/sample - loss: 0.2940 - acc: 0.8709 - val_loss: 0.5169 - val_acc: 0.7931
E

In [29]:
model.save_weights("trainings/model.h5")